## Exercise A - LHCb: First Observation of the Rare Decay $ B^0 \to p\bar{p} $

In [1]:
import ROOT

In [2]:
# declare the invariant mass observable 'm'
m = ROOT.RooRealVar("m", "m(p\\bar{p}) [\\mathrm{MeV/c^{2}}]", 5100, 5575)

# load the unbinned dataset
data = ROOT.RooDataSet.read("rarest_b0_decay.dat", ROOT.RooArgList(m), "v")

[#1] INFO:DataHandling -- RooDataSet::read: reading file rarest_b0_decay.dat
[#1] INFO:InputArguments -- RooRealVar::isValid(m): value 5096.96 out of range (5100 - 5575)
[#1] INFO:InputArguments -- RooRealVar::isValid(m): value 5088.49 out of range (5100 - 5575)
[#1] INFO:InputArguments -- RooRealVar::isValid(m): value 5095.88 out of range (5100 - 5575)
[#1] INFO:InputArguments -- RooRealVar::isValid(m): value 5096.17 out of range (5100 - 5575)
[#1] INFO:InputArguments -- RooRealVar::isValid(m): value 5098.32 out of range (5100 - 5575)
[#1] INFO:InputArguments -- RooRealVar::isValid(m): value 5084.75 out of range (5100 - 5575)
[#1] INFO:InputArguments -- RooRealVar::isValid(m): value 5087.37 out of range (5100 - 5575)
[#1] INFO:DataHandling -- RooDataSet::read: read 159 events (ignored 7 out of range events)


In [3]:
# visualize the data
c = ROOT.TCanvas()
frame = m.frame()
frame.SetTitle("Invariant Mass Distribution")
data.plotOn(frame)
frame.Draw()
c.Draw()

In [4]:
### Extended composite model for the invariant mass ###

# define a background component: exponential 
# define tau with an initial value of 20 as a free parameter
tau = ROOT.RooRealVar("tau", "tau", 400, 200, 2000)
# define the rate/alpha parameter of the exponential curve 
rate = ROOT.RooFormulaVar("rate", "exp rate", "-1./tau", [tau])
bkg = ROOT.RooExponential("bkg", "Background Component", m, rate)

# define a Gaussian PDF around the B^0 mass = 5279.72 MeV
mean1 = ROOT.RooRealVar("mean1", "mean of gaussian 1", 5279.72)
mean1.setConstant(True) 
sigma1 = ROOT.RooRealVar("sigma1", "width of gaussian 1", 20, 12, 35)
b0 = ROOT.RooGaussian("b0", "b0", m, mean1, sigma1)

# define a Gaussian PDF around the B^0_s mass = 5366.93 MeV
mean2 = ROOT.RooRealVar("mean2", "mean of gaussian 2", 5366.93)
mean2.setConstant(True)
#sigma2 = ROOT.RooRealVar("sigma2", "width of gaussian 2", 10, 0.1, 100)
b0s = ROOT.RooGaussian("b0s", "b0s", m, mean2, sigma1)

In [5]:
# define the expected number of events for background and signal components
nsig1 = ROOT.RooRealVar("nsig1", "Expected number of b0", 150, 50, 500)
nsig2 = ROOT.RooRealVar("nsig2", "Expected number of b0s", 5, 0, 30)
nbkg = ROOT.RooRealVar("nbkg", "Expected number of background events", 200, 50, 600)

# define the composite model
#   model = nsig1*b0 + nsig2*b0s + nbkg*bkg
model = ROOT.RooAddPdf("model", "Composite model", ROOT.RooArgList(b0, b0s, bkg), ROOT.RooArgList(nsig1, nsig2, nbkg))

In [6]:
# set number of bins to 25
m.setBins(25)

### Fit the model using Maximum Likelihood ###
model.fitTo(data, ROOT.RooFit.Extended(True), ROOT.RooFit.PrintLevel(-1))

nsig1.Print()
nsig2.Print()
nbkg.Print()

c2 = ROOT.TCanvas("c2", "c2")
frame2 = m.frame()
frame2.SetTitle("Invariant Mass Distribution")

data.plotOn(frame2, ROOT.RooFit.Name("Data"))
model.plotOn(frame2, ROOT.RooFit.Name("Fit"))
model.plotOn(frame2, ROOT.RooFit.Components("b0"), ROOT.RooFit.Name("b0"),
                ROOT.RooFit.LineColor(ROOT.kRed), ROOT.RooFit.LineStyle(ROOT.kDashed))
model.plotOn(frame2, ROOT.RooFit.Components("b0s"), ROOT.RooFit.Name("b0s"),
                ROOT.RooFit.LineColor(ROOT.kViolet), ROOT.RooFit.LineStyle(ROOT.kDashed))
model.plotOn(frame2, ROOT.RooFit.Components("bkg"), ROOT.RooFit.Name("bkg"),
                ROOT.RooFit.LineColor(ROOT.kGreen+2), ROOT.RooFit.LineStyle(ROOT.kDotted))

frame2.GetYaxis().SetRangeUser(0, 25)
frame2.GetXaxis().SetRangeUser(5100, 5550)

frame2.Draw()

legend = ROOT.TLegend(0.65, 0.7, 0.85, 0.85)
legend.AddEntry(frame2.findObject("Data"), "Data", "lep")
legend.AddEntry(frame2.findObject("Fit"), "Extended ML Fit", "l")
legend.AddEntry(frame2.findObject("b0"), "\\mathrm{B^{0}} \\rightarrow p \\bar{p}", "l")
legend.AddEntry(frame2.findObject("b0s"), "\\mathrm{B^{0}_{s}} \\rightarrow p \\bar{p}", "l")
legend.AddEntry(frame2.findObject("bkg"), "Comb. bkg.", "l")

legend.Draw()
c2.Draw()

[#1] INFO:Fitting -- RooAbsPdf::fitTo(model) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_dataset) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: deactivating const optimization
RooRealVar::nsig1 = 50.0003 +/- 3.17628  L(50 - 500) 
RooRealVar::nsig2 = 6.36488 +/- 4.66101  L(0 - 30) 
RooRealVar::nbkg = 110.468 +/- 12.1468  L(50 - 600) 
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) directly selected PDF components: (b0)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) indirectly selected PDF components: ()
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) directly selected PDF components: (b0s)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) indirectly selecte

In [7]:
### Create histograms of residuals and pulls ###
# calculate chi^2
print("Chi-suqared:", frame2.chiSquare())

Chi-suqared: 1.5871021492787294


In [8]:
# construct a histogram with the residuals of the data w.r.t. the curve
hresid = frame2.residHist()

# construct a histogram with the pulls of the data w.r.t. the curve
hpull = frame2.pullHist()

In [9]:
# create a frame to draw the residual distirbution and add the
# distribution to the frame
frame3 = m.frame()
frame3.SetTitle("Residual Distribution")
frame3.addPlotable(hresid, "P")

# create a frame to draw the pull distribution and add the distribution 
# to the frame
frame4 = m.frame()
frame4.SetTitle("Pull Distribution")
frame4.addPlotable(hpull, "P")

In [10]:
c3 = ROOT.TCanvas("c3", "c3", 900, 300)
c3.Divide(3)

c3.cd(1)
ROOT.gPad.SetLeftMargin(0.15)
frame2.GetYaxis().SetTitleOffset(1.4)
frame2.Draw()

c3.cd(2)
ROOT.gPad.SetLeftMargin(0.15)
frame3.GetYaxis().SetTitleOffset(1.4)
frame3.Draw()

c3.cd(3)
ROOT.gPad.SetLeftMargin(0.15)
frame4.GetYaxis().SetTitleOffset(1.4)
frame4.Draw()

c3.Draw()